# 02 Search query processing (Immigration)

This script processes the ___immigration___ search queries:
1. Data cleaning
2. Descriptive variables
3. Extract terms
4. Prepare data for manual coding

TO DO: what kind data comes out??

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import re
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
from collections import Counter
from statistics import mean, median
import nltk
from nltk.corpus import stopwords
from datetime import datetime
date = datetime.now().strftime('%d%m%Y')

/Users/marieke/Library/Python/3.8/lib/python/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#pd.set_option('max_columns', None)
#pd.set_option('max_rows', None)

In [4]:
#PATH = "/Users/marieke/surfdrive/Study1/data/"

In [5]:
# setting paths
PATH = '/Users/marieke/SearchingForBias'

In [7]:
# load search query data set
df = pd.read_pickle(PATH+"/data/immigration/01_df_sq.pkl")
df.shape

(1994, 4)

In [8]:
df.set_index('ID', inplace=True)
#df.head()

### 1. Data cleaning

In [9]:
# data cleaning functions
def lower_punc(x):
    x = re.sub(r'[^\w\s]|_', '', x.lower()).strip()
    return x

EXCLUDE = ["", "geen", "nvt", "niets", "geen idee", "niks", "weet niet", "niet", "idem"]
def remove_exclude(x):
    x = ' '.join([x for x in x.lower().split() if x not in EXCLUDE])
    return x

def remove_stopwords(x):
    x = [w for w in x.lower() if w not in stopwords.words('dutch')]
    return x

In [10]:
# lowercase + punctuation removal
cols_lp = [str(c)+"_lp" for c in df.columns]
for old, new in zip(df.columns, cols_lp):
    df[new] = df[old].apply(lower_punc)

In [11]:
# exclude uninformative words
cols_ex = [str(c)+"_ex" for c in cols_lp]
for old, new in zip(cols_lp, cols_ex):
    df[new] = df[old].apply(remove_exclude)

In [13]:
df.head()

,MVH_search_im_1,MVH_search_im_2,MVH_search_im_3,MVH_search_im_1_lp,MVH_search_im_2_lp,MVH_search_im_3_lp,MVH_search_im_1_lp_ex,MVH_search_im_2_lp_ex,MVH_search_im_3_lp_ex
ID,,,,,,,,,
1,data criminaliteit allochtonen,immigratie data,criminaliteit demografie,data criminaliteit allochtonen,immigratie data,criminaliteit demografie,data criminaliteit allochtonen,immigratie data,criminaliteit demografie
2,asielzoekers,alochtonen,asielzoekerscentrum,asielzoekers,alochtonen,asielzoekerscentrum,asielzoekers,alochtonen,asielzoekerscentrum
3,Immigranten in Nederland laatste 5 jaar CBS,Reden van vluchten immigranten,Refugees in Europe,immigranten in nederland laatste 5 jaar cbs,reden van vluchten immigranten,refugees in europe,immigranten in nederland laatste 5 jaar cbs,reden van vluchten immigranten,refugees in europe
4,Geen,Geen,Geen,geen,geen,geen,,,
5,reintegratie vluchtelingen,asielzoekers Nederland,vluchtelingen,reintegratie vluchtelingen,asielzoekers nederland,vluchtelingen,reintegratie vluchtelingen,asielzoekers nederland,vluchtelingen


### 2. Descriptive variables

In [14]:
def get_nwords(x):
    count = len(x.split())
    return count

def get_length(x):
    count = len(x)
    return count

In [15]:
# number of words
c_nwords = []
for c in cols_lp:
    name = str(c)+"_nwords"
    df[name] = df[c].apply(get_nwords)
    c_nwords.append(name)
    
df['im_nwords_min'] = df[c_nwords].min(axis=1)
df['im_nwords_max'] = df[c_nwords].max(axis=1)
df['im_nwords_mean'] = df[c_nwords].mean(axis=1)

In [16]:
df[c_nwords+['im_nwords_min', 'im_nwords_max', 'im_nwords_mean']].head()

,MVH_search_im_1_lp_nwords,MVH_search_im_2_lp_nwords,MVH_search_im_3_lp_nwords,im_nwords_min,im_nwords_max,im_nwords_mean
ID,,,,,,
1,3,2,2,2,3,2.333333
2,1,1,1,1,1,1.000000
3,7,4,3,3,7,4.666667
4,1,1,1,1,1,1.000000
5,2,2,1,1,2,1.666667


In [17]:
# length of search query
c_len = []
for c in cols_lp:
    name = str(c)+"_len"
    df[name] = df[c].apply(get_length)
    c_len.append(name)
    
df['im_len_min'] = df[c_len].min(axis=1)
df['im_len_max'] = df[c_len].max(axis=1)
df['im_len_mean'] = df[c_len].mean(axis=1)

In [18]:
df[c_len+['im_len_min', "im_len_max", "im_len_mean"]].head()

,MVH_search_im_1_lp_len,MVH_search_im_2_lp_len,MVH_search_im_3_lp_len,im_len_min,im_len_max,im_len_mean
ID,,,,,,
1,30,15,24,15,30,23.000000
2,12,10,19,10,19,13.666667
3,43,30,18,18,43,30.333333
4,4,4,4,4,4,4.000000
5,26,22,13,13,26,20.333333


In [19]:
#df['im_nwords_mean'].describe()

### 3. EXTRACT TERMS

In [29]:
# terms to be extracted
terms = ['immigrant', 'buitenlander', 'gelukszoeker', 'statushouder', 'allochtoon', 'vluchteling', 'expat', 'asielzoeker']

In [30]:
all_sq = [item for sublist in df[cols_lp].values.tolist() for item in sublist]
u_sq = set(all_sq)
all_words = [item for sublist in all_sq for item in sublist.split()]
u_words = set(all_words)

In [31]:
# how many search queries total?
len(all_sq)

5982

In [32]:
# To inform term dictionary with similar words --> all words with similarity >=80
sim_dict = {}
for t in terms:
    lst = [w for w in u_words if fuzz.token_sort_ratio(t, w)>=80]
    sim_dict.update({t:lst})

In [33]:
for k, v in sim_dict.items():
    print(k, v)

immigrant ['migrante', 'immigraten', 'immmigratie', 'immigrant', 'migrants', 'immigratie', 'immigranten', 'immigrattue', 'immigratis', 'inmmigratie', 'immigrante', 'imigranten', 'immigrantem', 'immigrants', 'immigraite', 'immigtanten', 'migrant', 'immigration']
buitenlander ['buitenlandse', 'buitenlanderd', 'buitenlnders', 'nuitenlandets', 'buitenlanders', 'buiytenland', 'buitelands', 'buitelanders', 'buitenlanďers', 'buitelanderse', 'buitenlander', 'buitenlands', 'buitenland', 'buitenanders']
gelukszoeker ['gelukzoekers', 'gelukszoekers', 'gelukszoekertjes', 'gelukzoelers']
statushouder ['statushouderd', 'statushouder', 'statushouders']
allochtoon ['autochtoon', 'allochtone', 'alochtonen', 'alloctonen', 'allochtonen', 'alochtoon', 'allochtoon']
vluchteling ['vluchteljngen', 'nepvluchtelingen', 'vluchten', 'vluvhteling', 'vlichtelingen', 'vluchteling', 'vluchtelinhen', 'vluchteingen', 'vluchtelinge', 'bootvluchteling', 'vluchtlingen', 'vluvhtelingen', 'vluchtenlingen', 'vluchtelingen',

In [34]:
# Nearly all words (excluding words that do not refer to a person, e.g."buitenland", "immigratie", etc.)
term_dict = {"immigrant":['migrants','immigtanten','migrante','immigraten','immigrant','immigrante','immigrantem',
                          'imigranten','immigrants','migrant','immigranten'],
             "buitenlander":['buitenlanďers','buitenlanderd','buitelanders','buitenlnders','buitenlander','buiytenland', 
                             'buitenanders','buitenlanders'],
             "gelukszoeker":['gelukszoekertjes','gelukszoekers','gelukzoelers','gelukzoekers'],
             "statushouder":['statushouderd', 'statushouder', 'statushouders'],
             "allochtoon":['allochtone','alochtonen','allochtoon','allochtonen','alochtoon', 'alloctonen'],
             "vluchteling":['vluvhteling','bootvluchteling','vluchtlingen','vluchtelngen','vluchtingen','vluchtelinge',
                            'vluchtenlingen','vluchtelinhen','vluchteljngen','vlichtelingen','vluchteingen','nepvluchtelingen',
                            'vluchteling','vluvhtelingen','vluchtelingen'],
             "expat":['expats', 'expat']
            }

#special treatment for asielzoeker to exclude asielzoekerscentrum
asielzoeker = ['asielzoeker', 'asielzoekers', 'assielzoeker', 'asielzo9eker', 'azilzoekers', 'assielzoekers',
               'asielxoekers', 'asiezoeker', 'asielziekers', 'asiepzoeker', 'azielzoekers', 'azielzoeker', 'asiel zoeker']

In [35]:
df['total'] = df['MVH_search_im_1_lp']+" "+df['MVH_search_im_2_lp']+" "+df['MVH_search_im_3_lp']
total = df['total'].values.tolist()

In [36]:
# find words in search queries
for k, v in term_dict.items():
    pattern = "|".join(v)
    print(pattern)
    df[k] = df['total'].str.contains(pattern, regex=True).astype(int)

migrants|immigtanten|migrante|immigraten|immigrant|immigrante|immigrantem|imigranten|immigrants|migrant|immigranten
buitenlanďers|buitenlanderd|buitelanders|buitenlnders|buitenlander|buiytenland|buitenanders|buitenlanders
gelukszoekertjes|gelukszoekers|gelukzoelers|gelukzoekers
statushouderd|statushouder|statushouders
allochtone|alochtonen|allochtoon|allochtonen|alochtoon|alloctonen
vluvhteling|bootvluchteling|vluchtlingen|vluchtelngen|vluchtingen|vluchtelinge|vluchtenlingen|vluchtelinhen|vluchteljngen|vlichtelingen|vluchteingen|nepvluchtelingen|vluchteling|vluvhtelingen|vluchtelingen
expats|expat


In [37]:
pattern2 = "|".join([str(s)+r"\b" for s in asielzoeker])
print(pattern2)
df['asielzoeker'] = df['total'].str.contains(pattern2, regex=True).astype(int)

asielzoeker\b|asielzoekers\b|assielzoeker\b|asielzo9eker\b|azilzoekers\b|assielzoekers\b|asielxoekers\b|asiezoeker\b|asielziekers\b|asiepzoeker\b|azielzoekers\b|azielzoeker\b|asiel zoeker\b


In [38]:
# check
#df[['total']+terms].head(10)

In [39]:
# first query only
for k, v in term_dict.items():
    pattern = "|".join(v)
    #print(pattern)
    new = k+"_1"
    df[new] = df['MVH_search_im_1_lp'].str.contains(pattern, regex=True).astype(int)
#asielzoeker
df['asielzoeker_1'] = df['MVH_search_im_1_lp'].str.contains(pattern2, regex=True).astype(int)

migrants|immigtanten|migrante|immigraten|immigrant|immigrante|immigrantem|imigranten|immigrants|migrant|immigranten
buitenlanďers|buitenlanderd|buitelanders|buitenlnders|buitenlander|buiytenland|buitenanders|buitenlanders
gelukszoekertjes|gelukszoekers|gelukzoelers|gelukzoekers
statushouderd|statushouder|statushouders
allochtone|alochtonen|allochtoon|allochtonen|alochtoon|alloctonen
vluvhteling|bootvluchteling|vluchtlingen|vluchtelngen|vluchtingen|vluchtelinge|vluchtenlingen|vluchtelinhen|vluchteljngen|vlichtelingen|vluchteingen|nepvluchtelingen|vluchteling|vluvhtelingen|vluchtelingen
expats|expat


In [40]:
# third query only
for k, v in term_dict.items():
    pattern = "|".join(v)
    #print(pattern)
    new = k+"_3"
    df[new] = df['MVH_search_im_3_lp'].str.contains(pattern, regex=True).astype(int)
#asielzoeker
df['asielzoeker_3'] = df['MVH_search_im_3_lp'].str.contains(pattern2, regex=True).astype(int)

migrants|immigtanten|migrante|immigraten|immigrant|immigrante|immigrantem|imigranten|immigrants|migrant|immigranten
buitenlanďers|buitenlanderd|buitelanders|buitenlnders|buitenlander|buiytenland|buitenanders|buitenlanders
gelukszoekertjes|gelukszoekers|gelukzoelers|gelukzoekers
statushouderd|statushouder|statushouders
allochtone|alochtonen|allochtoon|allochtonen|alochtoon|alloctonen
vluvhteling|bootvluchteling|vluchtlingen|vluchtelngen|vluchtingen|vluchtelinge|vluchtenlingen|vluchtelinhen|vluchteljngen|vlichtelingen|vluchteingen|nepvluchtelingen|vluchteling|vluvhtelingen|vluchtelingen
expats|expat


In [41]:
# checks
#df[['MVH_search_im_3_lp']+[c+"_3" for c in term_dict.keys()]]
#df[['MVH_search_im_1_lp']+[c+"_1" for c in term_dict.keys()]]

In [43]:
# save new variables.
df.to_pickle(PATH+'/data/immigration/02_df_sq_vars.pkl')

### 4. Manual annotation prep
- Make list of unique search queries to annotate, export as csv
- Make list of matches for reference

In [44]:
def fuzzy_matching(sq_list, threshold=100): 
    '''Takes a list of words. Fuzzy matches search queries
    (fuzz.token_sort_ratio) with similarity above treshold (default=100) and creates (a) a list of 
    unique search queries and (b) a dict including the search query matches (key = search query in annotation list, 
    values = matches not included in list)'''
    
    # Note. This method is considerably slow when using python default SequenceMatcher.
    # Install python-Levenshtein to speed things up (though install issues on macs). 
    
    sq_list = [w for w in set(sq_list) if w not in EXCLUDE]
    
    to_annotate = sq_list.copy()
    keys=[]
    vals=[]
    for i, element in enumerate(tqdm(sq_list)):
        if not any(element in sublist for sublist in vals):
            v_list = []
            for choice in sq_list[i+1:]:
                if fuzz.token_sort_ratio(element, choice)>=threshold:
                    v_list.append(choice)
      
            if v_list:
                vals.append(v_list)
                keys.append(element)
    lookup = dict(zip(keys,vals))
    
    remove = [x for v in lookup.values() for x in v]
    to_annotate = [x for x in to_annotate if x not in remove]
    
    print('Number of unique search queries to annotate:', len(to_annotate),
         '\nNumber of doubles removed:', len(remove))
    return to_annotate, lookup

In [45]:
unique, matches = fuzzy_matching(list(u_sq))

100%|██████████| 2802/2802 [06:35<00:00,  7.08it/s] 

Number of unique search queries to annotate: 2743 
Number of doubles removed: 59


In [46]:
# save to csv
pd.DataFrame(unique, columns=["search query"]).to_csv(PATH+f'/data/immigration/manual_coding/sq_to_annotate_{date}.csv', sep=";", index=False)

In [47]:
#store matches
with open(PATH+f'/data/immigration/manual_coding/sq_matches_{date}.txt', 'w') as outfile:
    json.dump(matches, outfile)

In [48]:
matches

{'immigratie regels': ['regels immigratie'],
 'nederland integratie': ['integratie nederland'],
 'nieuws immigranten': ['immigranten nieuws'],
 'immigratie nederland': ['nederland immigratie'],
 'immigranten criminaliteit': ['immigranten  criminaliteit'],
 'buitenlanders  nederland': ['buitenlanders nederland'],
 'ind beleid': ['beleid ind'],
 'verhuizen naar nederlands': ['naar nederlands verhuizen'],
 'cijfers migratie nederland': ['migratie cijfers nederland'],
 'nieuws immigratie nederland': ['immigratie nederland nieuws'],
 'asielzoekers aantallen': ['aantallen asielzoekers'],
 'herkomst immigranten': ['immigranten herkomst'],
 'europa immigratie': ['immigratie europa'],
 'immigratie beleid': ['beleid immigratie'],
 'verblijfsvergunning aanvragen': ['aanvragen verblijfsvergunning'],
 'vluchtelingen  nederland': ['vluchtelingen nederland',
  'nederland vluchtelingen'],
 'migratie cbs': ['cbs migratie'],
 'cijfers migratie': ['migratie cijfers'],
 'regels immigranten': ['immigranten